# Práctica 2: Optimización mediante Metahuerísticas

En la Práctica 1 se estudiaron métodos de búsqueda para resolver problemas que se pueden representar como un espacio de estados. La particularidad de dichos problemas era que el objetivo era doble: encontrar la mejor solución y encontrar la ruta desde el estado inicial al estado final.

En el mundo real existen problemas que solo es necesario encontrar la mejor solución y no el conjunto de estados desde el inicial al final. Estos problemas se conocen como problemas de optimización. Para resolver estos problemas existen distintos tipos de métodos: búsqueda aleatoria, búsqueda local (trayectorias) y basado en poblaciones.

### 8 Reinas

A continuación vamos a ver cómo se resuelve un problema de la práctica 1, el de las 8 reinas, por métodos basados en busqueda sin información y búsqueda local.

Primeramente vamos a definir el problema.

In [1]:
import sys
from search import Problem, Node
from utils4e import *
from collections import deque

class NQueensProblem(Problem):
    """The problem of placing N queens on an NxN board with none attacking
    each other. A state is represented as an N-element array, where
    a value of r in the c-th entry means there is a queen at column c,
    row r, and a value of -1 means that the c-th column has not been
    filled in yet. We fill in columns left to right.
    >>> depth_first_tree_search(NQueensProblem(8))
    <Node (7, 3, 0, 2, 5, 1, 6, 4)>
    """

    def __init__(self, N):
        super().__init__(tuple([-1] * N))
        self.N = N

    def actions(self, state):
        """In the leftmost empty column, try all non-conflicting rows."""
        if state[-1] != -1:
            return []  # All columns filled; no successors
        else:
            col = state.index(-1)
            return [row for row in range(self.N)
                    if not self.conflicted(state, row, col)]

    def result(self, state, row):
        """Place the next queen at the given row."""
        col = state.index(-1)
        new = list(state[:])
        new[col] = row
        return tuple(new)

    def conflicted(self, state, row, col):
        """Would placing a queen at (row, col) conflict with anything?"""
        return any(self.conflict(row, col, state[c], c)
                   for c in range(col))

    def conflict(self, row1, col1, row2, col2):
        """Would putting two queens in (row1, col1) and (row2, col2) conflict?"""
        return (row1 == row2 or  # same row
                col1 == col2 or  # same column
                row1 - col1 == row2 - col2 or  # same \ diagonal
                row1 + col1 == row2 + col2)  # same / diagonal

    def goal_test(self, state):
        """Check if all columns filled, no conflicts."""
        if state[-1] == -1:
            return False
        return not any(self.conflicted(state, state[col], col)
                       for col in range(len(state)))

    def h(self, node):
        """Return number of conflicting queens for a given node"""
        num_conflicts = 0
        for (r1, c1) in enumerate(node.state):
            for (r2, c2) in enumerate(node.state):
                if (r1, c1) != (r2, c2):
                    num_conflicts += self.conflict(r1, c1, r2, c2)

        return num_conflicts
    
    def value(self, state):
        num_conflicts = 0
        for (r1, c1) in enumerate(state):
            for (r2, c2) in enumerate(state):
                if (r1, c1) != (r2, c2):
                    num_conflicts += self.conflict(r1, c1, r2, c2)

        return num_conflicts

Código para crear el problema que se resolverá con distintos métodos.

In [2]:
problem = NQueensProblem(8)

Código para imprimir el tablero

In [3]:
def print_result(tlabel, x):
    
    print(tlabel+"\n")
    
    
    board = [["0"]*len(x) for i in range(len(x))]
    for c,r in enumerate(x):
        board[r][c] = "1"
    
    print("\n".join(" ".join(row) for row in board))

#### Búsqueda sin información
##### Búsqueda en profundidad

In [4]:
def depth_first_tree_search(problem):
    """
    [Figure 3.7]
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """

    frontier = [Node(problem.initial)]  # Stack

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None

In [5]:
depth_result = depth_first_tree_search(problem)
print_result("Profundidad", depth_result.state)

Profundidad

0 0 1 0 0 0 0 0
0 0 0 0 0 1 0 0
0 0 0 1 0 0 0 0
0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 1
0 0 0 0 1 0 0 0
0 0 0 0 0 0 1 0
1 0 0 0 0 0 0 0


##### Búsqueda en anchura

In [6]:
def breadth_first_tree_search(problem):
    """
    [Figure 3.7]
    Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Repeats infinitely in case of loops.
    """

    frontier = deque([Node(problem.initial)])  # FIFO queue

    while frontier:
        node = frontier.popleft()
        if problem.is_goal(node.state):
            return node
        frontier.extend(node.expand(problem))
    return None

In [7]:
breadth_result=breadth_first_tree_search(problem)
print_result("Anchura", depth_result.state)

Anchura

0 0 1 0 0 0 0 0
0 0 0 0 0 1 0 0
0 0 0 1 0 0 0 0
0 1 0 0 0 0 0 0
0 0 0 0 0 0 0 1
0 0 0 0 1 0 0 0
0 0 0 0 0 0 1 0
1 0 0 0 0 0 0 0


#### Búsqueda Local
##### Hill Climbing

In [8]:
def hill_climbing_min(problem):
    """
    [Figure 4.2]
    From the initial node, keep choosing the neighbor with lowest value,
    stopping when no neighbor is better.
    """
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmin_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) >= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [9]:
hill_result = hill_climbing_min(problem)
print_result("Hill", hill_result)

Hill

0 0 0 1 0 0 0 0
0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0
0 0 0 0 0 0 1 0
0 0 0 0 1 0 0 0
0 0 1 0 0 0 0 0
1 0 0 0 0 0 0 1


¿Se ha quedado en un óptimo local?

In [10]:
def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * np.exp(-lam * t) if t < limit else 0)


def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(np.exp(delta_e / T)):
            current = next_choice

In [11]:
ann_result = simulated_annealing(problem)
print_result("Simulated", ann_result)

Simulated

1 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0
0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0
0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0
0 0 0 1 0 0 1 1


##### Créditos
El código empleado en esta explicación está basado en el Libro titulado "Artificial Intelligence: A Modern Approach" y cuyo repositorio principal de código está en [https://github.com/aimacode/aima-python](https://github.com/aimacode/aima-python)